In [5]:

import csv, io, re, hashlib
from typing import List
import pandas as pd

try:
    from bs4 import BeautifulSoup
    HAVE_BS4 = True
except Exception:
    HAVE_BS4 = False

def sniff_delimiter(sample: str) -> str:
    """Very simple delimiter sniff: tab vs comma."""
    return "\t" if sample.count("\t") > sample.count(",") else ","

def html_to_text(html: str) -> str:
    """Convert HTML to plain text. Falls back if bs4 is missing."""
    if not html:
        return ""
    if HAVE_BS4:
        soup = BeautifulSoup(html, "html.parser")
        text = soup.get_text("\n", strip=True)
        text = re.sub(r"[ \t]+\n", "\n", text)
        text = re.sub(r"\n{3,}", "\n\n", text)
        return text
    # Fallback: crude stripping
    text = re.sub(r"<br\s*/?>", "\n", html, flags=re.I)
    text = re.sub(r"</p\s*>", "\n", text, flags=re.I)
    text = re.sub(r"<[^>]+>", "", text)
    text = text.replace("&nbsp;", " ").replace("&amp;", "&")
    text = re.sub(r"[ \t]+\n", "\n", text).strip()
    return text

def parse_url_list(cell: str) -> List[str]:
    """Split the images cell into a de-duplicated, ordered list of URLs."""
    if not cell:
        return []
    parts = [p.strip() for p in cell.split(",")]
    seen = set()
    out: List[str] = []
    for p in parts:
        if not p or not p.startswith("http"):
            continue
        if p not in seen:
            seen.add(p)
            out.append(p)
    return out

def url_hash(url: str, n: int = 16) -> str:
    """Stable short id for an image based on its URL."""
    return hashlib.sha256(url.encode("utf-8")).hexdigest()[:n]


In [6]:

from pathlib import Path

CONTEXT_FIELDS = [
    "context_id","date","name","description","latitude","longitude",
    "type_code","category","minimum period","maximum period",
    "creator","reviewer","reviewer_notes_text"
]
IMAGES_FIELDS = ["image_id","context_id","image_index","image_url"]

def split_oervondstchecker(input_path: str,
                           context_out: str = "oervondstchecker_context.csv",
                           images_out: str = "oervondstchecker_images.csv") -> None:
    """Split the export into two CSVs (context + images)."""
    path = Path(input_path)
    if not path.exists():
        raise FileNotFoundError(f"Input file not found: {path}")
    sample = path.read_text(encoding="utf-8", errors="ignore")[:4096]
    delim = sniff_delimiter(sample)

    # Read robustly with DictReader
    rows = []
    with path.open("r", encoding="utf-8", newline="") as f:
        reader = csv.DictReader(f, delimiter=delim, quotechar='"')
        for r in reader:
            rows.append(r)

    # Build outputs
    ctx_records = []
    img_records = []
    context_id = 0
    for r in rows:
        context_id += 1
        ctx_records.append({
            "context_id": context_id,
            "date": r.get("date",""),
            "name": r.get("name",""),
            "description": r.get("description",""),
            "latitude": r.get("latitude",""),
            "longitude": r.get("longitude",""),
            "type_code": r.get("type_code",""),
            "category": r.get("category",""),
            "minimum period": r.get("minimum period",""),
            "maximum period": r.get("maximum period",""),
            "creator": r.get("creator",""),
            "reviewer": r.get("reviewer",""),
            "reviewer_notes_text": html_to_text(r.get("reviewer_notes","")),
        })
        urls = parse_url_list(r.get("images",""))
        seen = set()
        idx = 0
        for u in urls:
            if u in seen:
                continue
            seen.add(u)
            idx += 1
            img_records.append({
                "image_id": url_hash(u),
                "context_id": context_id,
                "image_index": idx,
                "image_url": u,
            })

    # Write CSVs
    df_ctx = pd.DataFrame(ctx_records, columns=CONTEXT_FIELDS)
    df_img = pd.DataFrame(img_records, columns=IMAGES_FIELDS)
    df_ctx.to_csv(context_out, index=False)
    df_img.to_csv(images_out, index=False)
    print(f"Wrote {context_out} ({len(df_ctx)} rows)")
    print(f"Wrote {images_out} ({len(df_img)} rows)")


In [ ]:
# ==== Parameters ====
INPUT_PATH = "2023-01-13Data-exportOervondstchecker.csv" 
CONTEXT_OUT = "oervondstchecker_context.csv"
IMAGES_OUT = "oervondstchecker_images.csv"


In [8]:

# Run the split (make sure INPUT_PATH is correct)
try:
    split_oervondstchecker(INPUT_PATH, CONTEXT_OUT, IMAGES_OUT)
except FileNotFoundError as e:
    print(e)
    print("Place your export next to this notebook or update INPUT_PATH above.")

# Optional: Preview heads if files exist
try:
    import pandas as pd
    display(pd.read_csv(CONTEXT_OUT).head())
    display(pd.read_csv(IMAGES_OUT).head())
except Exception as e:
    pass


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x92 in position 922: invalid start byte